In [15]:
import lime
import numpy as np
import pandas as pd
import pandas_profiling
import sklearn
import sklearn.ensemble
import sklearn.metrics
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('charity.csv')
data.head()

,ID,reg1,reg2,reg3,reg4,home,chld,hinc,genf,wrat,...,npro,tgif,lgif,rgif,tdon,tlag,agif,donr,damt,part
0,1,0,0,1,0,1,1,4,1,8,...,20,81,81,19,17,6,21.05,0.0,0.0,train
1,2,0,0,1,0,1,2,4,0,8,...,95,156,16,17,19,3,13.26,1.0,15.0,train
2,3,0,0,1,0,1,1,5,1,8,...,64,86,15,10,22,8,17.37,NaN,NaN,test
3,4,0,0,0,0,1,1,4,0,8,...,51,56,18,7,14,7,9.59,NaN,NaN,test
4,5,0,0,1,0,1,0,4,1,4,...,85,132,15,10,10,6,12.07,1.0,17.0,valid


In [ ]:
np.random.seed(0)

In [3]:
pandas_profiling.ProfileReport(data)

Number of variables,24
Number of observations,8009
Total Missing (%),1.0%
Total size in memory,1.5 MiB
Average record size in memory,192.0 B
Numeric,16
Categorical,1
Boolean,6
Date,0
Text (Unique),0
Rejected,1


In [4]:
notest = data.loc[data['part'] != 'test']
nopart = notest.drop(columns=['part', 'ID', 'damt'])
nopart.shape

(6002, 21)

In [5]:
y = nopart.donr

In [6]:
nopart.pop('donr')
nopart.shape

(6002, 20)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(nopart, y, test_size=0.3)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4201, 20) (4201,)
(1801, 20) (1801,)


In [8]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [9]:
print(rf.feature_importances_)

[ 0.01209432  0.05810368  0.00505673  0.00664265  0.05974709  0.27047857
  0.09342455  0.00592376  0.06256788  0.04068578  0.0425366   0.03619295
  0.0330914   0.04456901  0.04854586  0.02958999  0.02920802  0.04994363
  0.03673565  0.03486188]


In [10]:
y_pred = rf.predict(X_test)
y_pred

array([ 1.,  0.,  1., ...,  0.,  1.,  1.])

In [18]:
y_pp = rf.predict_proba(X_test)
y_pp

array([[ 0.484,  0.516],
       [ 0.758,  0.242],
       [ 0.082,  0.918],
       ..., 
       [ 0.856,  0.144],
       [ 0.336,  0.664],
       [ 0.084,  0.916]])

In [11]:
print("Accuracy:", sklearn.metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.890061077179


In [72]:
sklearn.metrics.f1_score(y_test, y_pred, average='binary')

0.8923913043478261

In [12]:
pd.crosstab(y_test, y_pred)

col_0,0.0,1.0
donr,,
0.0,782,101
1.0,97,821


In [13]:
list(zip(X_train, rf.feature_importances_))

[('reg1', 0.012094319170080791),
 ('reg2', 0.05810368451319671),
 ('reg3', 0.0050567332706836603),
 ('reg4', 0.0066426549005545073),
 ('home', 0.059747087337055499),
 ('chld', 0.2704785725944393),
 ('hinc', 0.093424546485550361),
 ('genf', 0.005923760654618684),
 ('wrat', 0.062567875159267233),
 ('avhv', 0.040685782129324513),
 ('incm', 0.042536598987202814),
 ('inca', 0.036192951313133111),
 ('plow', 0.033091399520267767),
 ('npro', 0.044569009816483644),
 ('tgif', 0.048545860869553557),
 ('lgif', 0.029589994416427432),
 ('rgif', 0.029208020525129868),
 ('tdon', 0.049943626930654142),
 ('tlag', 0.036735645869808389),
 ('agif', 0.034861875536567839)]

In [65]:
from lime.lime_tabular import LimeTabularExplainer

In [68]:
explainer = LimeTabularExplainer(
    X_train.values, feature_names = X_train.columns)

In [75]:
exp = explainer.explain_instance(
    X_test.loc[64, :], rf[1].predict_proba, num_features=5
)

In [76]:
exp.show_in_notebook(show_table=True)